In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
from glob import glob # pour trouver les fichiers de tracking
import multiprocessing # pour le traitement parallèle
from functools import partial # pour faciliter l'utilisation de la fonction avec des arguments partiels
#import ffmpeg
from pyproj import Transformer
from scipy.optimize import linear_sum_assignment

### Les données LPS

In [2]:
fusion_3 = "../../DATABASE/basket/LPS/3_fusion.csv"
fusion_4 = "../../DATABASE/basket/LPS/4_fusion.csv"
fusion_5 = "../../DATABASE/basket/LPS/5_fusion.csv"
fusion_7 = "../../DATABASE/basket/LPS/7_fusion.csv"
fusion_10 = "../../DATABASE/basket/LPS/10_fusion.csv"
fusion_15 = "../../DATABASE/basket/LPS/15_fusion.csv"
fusion_16 = "../../DATABASE/basket/LPS/16_fusion.csv"
fusion_21 = "../../DATABASE/basket/LPS/21_fusion.csv"
fusion_22 = "../../DATABASE/basket/LPS/22_fusion.csv"
fusion_23 = "../../DATABASE/basket/LPS/23_fusion.csv"
fusion_29 = "../../DATABASE/basket/LPS/29_fusion.csv"

In [3]:
lps_3 = pd.read_csv(fusion_3, sep=";")
lps_4 = pd.read_csv(fusion_4, sep=";")
lps_5 = pd.read_csv(fusion_5, sep=";")
lps_7 = pd.read_csv(fusion_7, sep=";")
lps_10 = pd.read_csv(fusion_10, sep=";")
lps_15 = pd.read_csv(fusion_15, sep=";")
lps_16 = pd.read_csv(fusion_16, sep=";")
lps_21 = pd.read_csv(fusion_21, sep=";")
lps_22 = pd.read_csv(fusion_22, sep=";")
lps_23 = pd.read_csv(fusion_23, sep=";")
lps_29 = pd.read_csv(fusion_29, sep=";")

In [4]:
def lps_sample(df):
    df = df.copy()
    df = df[::2]# c'est tout ce qu'il fallait mettre pour échantillonner tous les 2 frames
    return df

def nettoyage_lps(df):
    columns_to_drop = ["lat_brute", "long_brute", "hdop", "vitesse_fusion", "battery"]
    df = df.drop(columns=columns_to_drop, errors='ignore')
    df = lps_sample(df)
    transformer = Transformer.from_crs("epsg:4326", "epsg:2154", always_xy=True)
    df['x'], df['y'] = transformer.transform(df['longitude_fusion'].values, df['latitude_fusion'].values)
    df['x_norm'] = df['x'] - df['x'].mean()
    df['y_norm'] = df['y'] - df['y'].mean()
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    start_time = df['datetime'].iloc[0]
    df['relative_time'] = (df['datetime'] - start_time).dt.total_seconds()
    df = df.dropna(subset=["latitude_fusion", "longitude_fusion"])
    df = df.reset_index(drop=True)
    return df

In [5]:
liste_lps = [lps_3, lps_4, lps_5, lps_7, lps_10, lps_15, lps_16, lps_21, lps_22, lps_23, lps_29]
for i in range(len(liste_lps)):
    liste_lps[i] = nettoyage_lps(liste_lps[i])

In [6]:
lps_dict = {i: df for i, df in enumerate(liste_lps)}

In [7]:
lps_dict[1]

,timestamp,latitude_fusion,longitude_fusion,x,y,x_norm,y_norm,datetime,relative_time
0,1702478113100,49.458017,1.062370,559484.458832,6.930458e+06,1.794312,-7.883335,2023-12-13 14:35:13.100,0.0
1,1702478113300,49.458014,1.062370,559484.428801,6.930458e+06,1.764281,-8.234861,2023-12-13 14:35:13.300,0.2
2,1702478113500,49.458011,1.062369,559484.362780,6.930458e+06,1.698260,-8.518783,2023-12-13 14:35:13.500,0.4
3,1702478113700,49.458009,1.062368,559484.271256,6.930458e+06,1.606736,-8.752733,2023-12-13 14:35:13.700,0.6
4,1702478113900,49.458008,1.062367,559484.194558,6.930457e+06,1.530038,-8.930941,2023-12-13 14:35:13.900,0.8
...,...,...,...,...,...,...,...,...,...
42339,1702486859900,49.458053,1.062344,559482.646752,6.930462e+06,-0.017768,-3.847196,2023-12-13 17:00:59.900,8746.8
42340,1702486860100,49.458054,1.062344,559482.675465,6.930463e+06,0.010945,-3.698696,2023-12-13 17:01:00.100,8747.0
42341,1702486860300,49.458056,1.062345,559482.714346,6.930463e+06,0.049826,-3.519262,2023-12-13 17:01:00.300,8747.2
42342,1702486860500,49.458058,1.062346,559482.776360,6.930463e+06,0.111840,-3.334244,2023-12-13 17:01:00.500,8747.4


Les fichiers lps ont été enregistrés de 14h35 a 17h01. Donc deux heures 25 min de jeu.

### Les données tracking

In [178]:
tracking_1_1_2_1 = "../../DATABASE/basket/TRACKING/Equipe1_Vague1_Poss2_video_1.txt"
tracking_df = pd.read_csv(tracking_1_1_2_1, names=['frame', 'player_id', 'x', 'y'], sep=",")

In [179]:
tracking_df

,frame,player_id,x,y
0,0,0,10.738531,2.423311
1,0,1,10.758081,7.556952
2,0,2,9.685260,7.294818
3,0,3,6.877824,9.319980
4,0,4,5.604179,9.292738
...,...,...,...,...
3282,385,3,5.606390,1.841755
3283,385,4,0.658973,5.352539
3284,385,6,2.336708,9.233820
3285,385,7,2.905753,8.886047


In [180]:
def tracking_sample(df):
    """
    Échantillonne le DataFrame en gardant une frame sur 5
    """
    df = df.copy()
    unique_frames = sorted(df['frame'].unique())
    selected_frames = unique_frames[::5]
    df = df[df['frame'].isin(selected_frames)]
    return df

def nettoyage_tracking(df):
    df = df.dropna(subset=["x", "y"])
    df = df.reset_index(drop=True)
    df = tracking_sample(df)
    df["time"] = df["frame"] / 25
    df["x"] = df["x"].astype(float)
    df["y"] = df["y"].astype(float)
    df["player_id"] = df["player_id"].astype(int)
    df["frame"] = df["frame"].astype(int)

    df["x_norm"] = df.groupby("player_id")["x"].transform(lambda x: (x - x.mean()) / x.std() if x.std() > 0 else 0)
    df["y_norm"] = df.groupby("player_id")["y"].transform(lambda y: (y - y.mean()) / y.std() if y.std() > 0 else 0)
    return df

In [181]:
tracking_df =nettoyage_tracking(tracking_df)
tracking_df.reset_index(drop=True)
tracking_df

,frame,player_id,x,y,time,x_norm,y_norm
0,0,0,10.738531,2.423311,0.0,-2.429077,-1.484317
1,0,1,10.758081,7.556952,0.0,0.883258,-1.001630
2,0,2,9.685260,7.294818,0.0,1.012336,-1.148542
3,0,3,6.877824,9.319980,0.0,0.268986,0.069098
4,0,4,5.604179,9.292738,0.0,1.425671,0.647658
...,...,...,...,...,...,...,...
3282,385,3,5.606390,1.841755,15.4,-0.775416,-1.917293
3283,385,4,0.658973,5.352539,15.4,-2.722745,-2.929088
3284,385,6,2.336708,9.233820,15.4,-2.354843,2.780589
3285,385,7,2.905753,8.886047,15.4,-2.241351,3.329381


In [182]:
joueur_0 = tracking_df[tracking_df["player_id"] == 0]
joueur_0 = joueur_0.reset_index(drop=True)
joueur_0[:20]


,frame,player_id,x,y,time,x_norm,y_norm
0,0,0,10.738531,2.423311,0.0,-2.429077,-1.484317
1,5,0,10.852108,2.521917,0.2,-2.294991,-1.458786
2,10,0,10.846599,2.646704,0.4,-2.301495,-1.426477
3,15,0,11.077645,2.852193,0.6,-2.028728,-1.373272
4,20,0,11.277779,3.008532,0.8,-1.792456,-1.332793
5,25,0,11.309185,3.103925,1.0,-1.755378,-1.308094
6,30,0,11.699689,3.401040,1.2,-1.294360,-1.231166
7,35,0,11.808855,3.543278,1.4,-1.165481,-1.194338
8,40,0,11.792461,3.678334,1.6,-1.184835,-1.159370
9,45,0,12.053783,3.952349,1.8,-0.876325,-1.088422


### Let's test something

### CREATION DE L'ANIMATION POUR LPS

In [56]:
def create_lps_animation(lps_df, start_time, duration=20, save_path="lps_animation.gif"):
    """
    Crée une animation GIF des positions LPS à partir d'un temps de début donné.

    Args:
        lps_df: DataFrame LPS avec colonnes ['relative_time', 'x', 'y']
        start_time: temps de début (en secondes, relatif)
        duration: durée de l'animation en secondes (par défaut 20)
        save_path: chemin du fichier GIF à sauvegarder
    """
    # Filtrer la fenêtre temporelle
    end_time = start_time + duration
    window = lps_df[(lps_df['relative_time'] >= start_time) & (lps_df['relative_time'] <= end_time)].copy()
    window = window.dropna(subset=['x_norm', 'y_norm'])
    if window.empty:
        print("Aucune donnée LPS dans la fenêtre temporelle spécifiée.")
        return

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_facecolor('black')
    ax.grid(True, alpha=0.3, color='white')
    ax.set_xlabel('Latitude', fontsize=12, color='white')
    ax.set_ylabel('Longitude', fontsize=12, color='white')
    ax.tick_params(colors='white')

    x_min, x_max = window['x_norm'].min() - 0.0001, window['x_norm'].max() + 0.0001
    y_min, y_max = window['y_norm'].min() - 0.0001, window['y_norm'].max() + 0.0001
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    title = ax.text(0.5, 1.05, 'Animation des Positions LPS',
                    transform=ax.transAxes, ha='center', va='bottom',
                    fontsize=16, fontweight='bold', color='white')

    time_text = ax.text(0.02, 0.98, '', transform=ax.transAxes,
                        fontsize=12, color='cyan', verticalalignment='top',
                        bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))

    dot, = ax.plot([], [], 'o', markersize=10, color='orange', markeredgecolor='white', markeredgewidth=2, zorder=10)
    trail, = ax.plot([], [], '-', alpha=0.7, linewidth=2, color='orange')

    times = window['relative_time'].values
    total_frames = len(times)
    interval = (duration * 1000) / total_frames if total_frames > 0 else 50

    def animate(frame):
        t = times[frame]
        current = window.iloc[frame]
        dot.set_data([current['x_norm']], [current['y_norm']])
        trail.set_data(window['x_norm'].values[:frame+1], window['y_norm'].values[:frame+1])
        time_text.set_text(f"t = {t:.2f} s")
        return dot, trail, time_text

    anim = FuncAnimation(fig, animate, frames=total_frames, interval=interval, blit=True, repeat=True)
    anim.save(save_path, writer='pillow', fps=total_frames/duration)
    plt.close(fig)
    print(f"Animation LPS sauvegardée sous {save_path}")

#lps_3 = nettoyage_lps(lps_3)
create_lps_animation(lps_dict[1], start_time=0.0, duration=60, save_path="../../DATABASE/animation/lps_3_joueur_track_3_test_04.gif")

Animation LPS sauvegardée sous ../../DATABASE/animation/lps_3_joueur_track_3_test_04.gif


### CREATION DE L'ANIMATION POUR LE TRACKING

In [24]:
def create_tracking_animation(df, duration=20, save_path="animation.gif"):
    """
    Crée une animation dynamique des données de tracking sans afficher les trajectoires et sauvegarde en GIF.

    Args:
        df: DataFrame avec les colonnes ['time', 'player_id', 'x', 'y']
        duration: Durée de l'animation en secondes
        save_path: Chemin du fichier GIF à sauvegarder
    """

    fig, ax = plt.subplots(figsize=(12, 8))
    fig.patch.set_facecolor('black')

    x_min, x_max = df['x'].min() - 1, df['x'].max() + 1
    y_min, y_max = df['y'].min() - 1, df['y'].max() + 1

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_facecolor('darkslategray')
    ax.grid(True, alpha=0.3, color='white')
    ax.set_xlabel('Position X', fontsize=12, color='white')
    ax.set_ylabel('Position Y', fontsize=12, color='white')
    ax.tick_params(colors='white')

    title = ax.text(0.5, 1.05, 'Animation des Positions des Joueurs',
                    transform=ax.transAxes, ha='center', va='bottom',
                    fontsize=16, fontweight='bold', color='white')

    time_text = ax.text(0.02, 0.98, '', transform=ax.transAxes,
                        fontsize=12, color='cyan', verticalalignment='top',
                        bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))

    unique_players = sorted(df['player_id'].unique())
    colors = plt.cm.tab10(np.linspace(0, 1, len(unique_players)))
    player_colors = dict(zip(unique_players, colors))

    player_dots = {}
    player_labels = {}

    for player_id in unique_players:
        dot, = ax.plot([], [], 'o', markersize=12,
                       color=player_colors[player_id],
                       markeredgecolor='white', markeredgewidth=2,
                       zorder=10)
        player_dots[player_id] = dot

        label = ax.text(0, 0, f'{player_id}', fontsize=8, ha='center', va='center',
                        color='white', fontweight='bold', zorder=11)
        player_labels[player_id] = label

    legend_elements = [plt.Line2D([0], [0], marker='o', color='w',
                                  markerfacecolor=player_colors[pid], markersize=8,
                                  label=f'Joueur {pid}', markeredgecolor='white')
                       for pid in unique_players]
    ax.legend(handles=legend_elements, loc='upper right', facecolor='black',
              edgecolor='white', labelcolor='white')

    times = sorted(df['time'].unique())
    total_frames = len(times)
    interval = (duration * 1000) / total_frames

    def animate(frame):
        current_time = times[frame]
        current_data = df[df['time'] == current_time]

        time_text.set_text(f'time: {current_time:.2f}')

        for player_id in unique_players:
            player_data = current_data[current_data['player_id'] == player_id]

            if not player_data.empty:
                x, y = player_data.iloc[0]['x'], player_data.iloc[0]['y']
                player_dots[player_id].set_data([x], [y])
                player_labels[player_id].set_position((x, y))
            else:
                player_dots[player_id].set_data([], [])
                player_labels[player_id].set_position((x_min-10, y_min-10))

        return list(player_dots.values()) + list(player_labels.values()) + [time_text]

    anim = FuncAnimation(fig, animate, frames=total_frames,
                         interval=interval, blit=True, repeat=True)

    # Sauvegarde en GIF
    anim.save(save_path, writer='pillow', fps=total_frames/duration)
    plt.close(fig)
    print(f"Animation sauvegardée sous {save_path}")

# Exemple d'utilisation :
#create_tracking_animation(tracking_df, duration=20, save_path="../../DATABASE/animation/tracking_animation_3.gif")


### TEST DE LSAP

In [200]:
lps_keys = list(lps_dict.keys())
N = len(lps_keys)
print(f"Nombre de DataFrames LPS : {N}")
print("Clés disponibles :", lps_keys)

Nombre de DataFrames LPS : 11
Clés disponibles : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


Les durées de temps de tracking

In [183]:
tracking_times = tracking_df["time"].sort_index().unique()
tracking_times[:10]
n_frames = len(tracking_times)
delta_t = tracking_times[-1] - tracking_times[0]
print(f"Nombre total de frames dans le DataFrame de tracking : {n_frames}")
print(f"Durée totale du tracking : {delta_t:.2f} secondes")

Nombre total de frames dans le DataFrame de tracking : 78
Durée totale du tracking : 15.40 secondes


In [184]:
tracking_grouped = tracking_df.groupby("player_id")
print(f"Nombre de joueurs uniques dans le DataFrame de tracking : {len(tracking_grouped)}")
player_ids = tracking_df["player_id"].unique()
print(f"Identifiants des joueurs : {player_ids}")
tracking_grouped.get_group(0).head(10)

Nombre de joueurs uniques dans le DataFrame de tracking : 9
Identifiants des joueurs : [0 1 2 3 4 5 6 7 8]


,frame,player_id,x,y,time,x_norm,y_norm
0,0,0,10.738531,2.423311,0.0,-2.429077,-1.484317
45,5,0,10.852108,2.521917,0.2,-2.294991,-1.458786
90,10,0,10.846599,2.646704,0.4,-2.301495,-1.426477
135,15,0,11.077645,2.852193,0.6,-2.028728,-1.373272
180,20,0,11.277779,3.008532,0.8,-1.792456,-1.332793
225,25,0,11.309185,3.103925,1.0,-1.755378,-1.308094
270,30,0,11.699689,3.401040,1.2,-1.294360,-1.231166
315,35,0,11.808855,3.543278,1.4,-1.165481,-1.194338
360,40,0,11.792461,3.678334,1.6,-1.184835,-1.159370
405,45,0,12.053783,3.952349,1.8,-0.876325,-1.088422


In [210]:
valid_player_ids = []
traj_list = []
print(f"Nombre total de frames dans le DataFrame de tracking : {n_frames}")
for pid in player_ids:
    traj = tracking_grouped.get_group(pid).sort_values("time")
    traj = traj[['time', 'x_norm', 'y_norm']].drop_duplicates("time")
    if len(traj) < 0.75 * n_frames:
        continue
    traj_interp = pd.DataFrame({'time': tracking_times})
    traj_interp = traj_interp.merge(traj, on='time', how='left')
    traj_interp['x_norm'] = traj_interp['x_norm'].interpolate(method='linear', limit_direction='both')
    traj_interp['y_norm'] = traj_interp['y_norm'].interpolate(method='linear', limit_direction='both')
    valid_player_ids.append(pid)
    traj_list.append(traj_interp[['x_norm', 'y_norm']].values)

Nombre total de frames dans le DataFrame de tracking : 78


In [193]:
print(f"Nombre de joueurs valides après interpolation : {len(valid_player_ids)}")

Nombre de joueurs valides après interpolation : 8


In [202]:
for i in range(len(traj_list)):
    print(f"Trajectoire du joueur {valid_player_ids[i]} : \n{i, traj_list[i][:10]}")

Trajectoire du joueur 1 : 
(0, array([[ 0.88325799, -1.00162978],
       [ 0.88309498, -0.98345122],
       [ 0.88567708, -0.98735822],
       [ 0.87806331, -0.97991852],
       [ 0.88195467, -0.84525886],
       [ 0.88160451, -0.81712058],
       [ 0.84987777, -0.71484097],
       [ 0.87220076, -0.80805795],
       [ 0.85514604, -0.80615595],
       [ 0.88435675, -0.83995231]]))
Trajectoire du joueur 2 : 
(1, array([[ 1.01233606, -1.14854192],
       [ 1.02324311, -1.15005087],
       [ 1.06053555, -1.00252652],
       [ 1.06505916, -0.84928448],
       [ 1.02161218, -1.09476539],
       [ 1.04067105, -0.99620067],
       [ 1.0399086 , -0.76025433],
       [ 1.11974364, -0.59851251],
       [ 1.08548807, -0.54334748],
       [ 1.05428788, -0.80598996]]))
Trajectoire du joueur 3 : 
(2, array([[0.26898562, 0.06909799],
       [0.40343327, 0.12544836],
       [0.38947888, 0.25582006],
       [0.38649771, 0.33557778],
       [0.23666404, 0.40676929],
       [0.4943883 , 0.49560771],
     

In [203]:
P = len(valid_player_ids)
tracking_tensor = np.stack(traj_list, axis=0)
print(f"Tenseur de tracking créé avec la forme : {tracking_tensor.shape}")

Tenseur de tracking créé avec la forme : (8, 78, 2)


In [209]:
cost_matrix = np.full((N, P), np.inf)
cost_matrix

array([[inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf]])

In [211]:
for i, (name, lps_df) in enumerate(lps_dict.items()):
        lps_times = lps_df['relative_time'].sort_values().unique()

        for t0 in lps_times:
            t_end = t0 + delta_t
            lps_window = lps_df[(lps_df['relative_time'] >= t0) & (lps_df['relative_time'] <= t_end)]

            if len(lps_window) != n_frames:
                continue

            lps_traj = lps_window[['x_norm', 'y_norm']].values  # (T, 2)

            for j in range(P):
                dists = np.linalg.norm(tracking_tensor[j] - lps_traj, axis=1)
                mean_dist = dists.mean()

                # Mettre à jour si c’est meilleur
                if mean_dist < cost_matrix[i, j]:
                    cost_matrix[i, j] = mean_dist

In [213]:
cost_matrix.shape

(11, 8)

In [216]:
df_cost = pd.DataFrame(cost_matrix, index=lps_keys, columns=valid_player_ids)
df_cost

,1,2,3,4,5,6,7,8
0,1.272574,1.199904,1.059849,1.158349,1.040888,1.063517,1.152486,1.064786
1,1.299239,1.351131,0.878414,1.031584,1.101827,1.072046,1.084273,0.945070
2,2.717771,2.705408,2.536949,2.519933,2.537725,2.642391,2.693449,2.519514
3,2.146714,2.143856,1.463146,1.523962,1.539278,2.256432,2.197359,1.449722
4,1.322451,1.237997,1.298686,1.142164,1.275779,0.884958,0.912914,1.325112
5,1.688617,1.701283,1.285417,1.257356,1.301119,1.614746,1.582202,1.282940
6,1.467772,1.547120,1.490366,1.106460,1.228174,1.020704,1.047382,1.463439
7,1.150418,1.063028,1.161639,0.821225,0.817873,0.650106,0.728012,1.065975
8,1.484907,1.419193,1.281403,1.098162,1.130954,1.121052,1.117170,1.320636
9,1.421091,1.387368,1.348321,1.065962,1.180593,1.369368,1.300383,1.364348


In [ ]:
row_ind, col_ind = linear_sum_assignment(cost_matrix)
matches = [(lps_keys[i], valid_player_ids[j]) for i, j in zip(row_ind, col_ind)]
matches_df = pd.DataFrame(matches, columns=['lps_file', 'player_id'])

           1         2         3         4         5         6         7  \
0   1.272574  1.199904  1.059849  1.158349  1.040888  1.063517  1.152486   
1   1.299239  1.351131  0.878414  1.031584  1.101827  1.072046  1.084273   
2   2.717771  2.705408  2.536949  2.519933  2.537725  2.642391  2.693449   
3   2.146714  2.143856  1.463146  1.523962  1.539278  2.256432  2.197359   
4   1.322451  1.237997  1.298686  1.142164  1.275779  0.884958  0.912914   
5   1.688617  1.701283  1.285417  1.257356  1.301119  1.614746  1.582202   
6   1.467772  1.547120  1.490366  1.106460  1.228174  1.020704  1.047382   
7   1.150418  1.063028  1.161639  0.821225  0.817873  0.650106  0.728012   
8   1.484907  1.419193  1.281403  1.098162  1.130954  1.121052  1.117170   
9   1.421091  1.387368  1.348321  1.065962  1.180593  1.369368  1.300383   
10  1.476545  1.390291  1.070923  1.140597  1.257032  1.192069  1.208241   

           8  
0   1.064786  
1   0.945070  
2   2.519514  
3   1.449722  
4   1.325112

In [217]:
matches_df['cost'] = cost_matrix[row_ind, col_ind]
print(matches_df)

   lps_file  player_id      cost
0         0          1  1.272574
1         1          3  0.878414
2         4          2  1.237997
3         6          7  1.047382
4         7          6  0.650106
5         8          5  1.130954
6         9          4  1.065962
7        10          8  1.039858
